In [ ]:
import httpx
import ssl

url = "https://wwwnew.kweather.co.kr/forecast/data/lifestyle/27230510.xml"

# SSL 설정: 보안 수준 낮춤
context = ssl.create_default_context()
context.set_ciphers("DEFAULT:@SECLEVEL=1")  # 보안 레벨 1로 낮춤

try:
    with httpx.Client(verify=context, timeout=10.0) as client:
        response = client.get(url)
        response.encoding = 'utf-8'
        print(response.text)
except Exception as e:
    print(f"Error: {e}")


In [ ]:
import httpx
import ssl
from bs4 import BeautifulSoup

field_list = ['48160720','27230510']

for code in field_list:
    url = f"https://wwwnew.kweather.co.kr/forecast/data/lifestyle/{code}.xml"

    # SSL 설정: 보안 수준 낮춤
    context = ssl.create_default_context()
    context.set_ciphers("DEFAULT:@SECLEVEL=1")  # 보안 레벨 1로 낮춤

    try:
        with httpx.Client(verify=context, timeout=10.0) as client:
            response = client.get(url)
            response.encoding = 'utf-8'
            #print(response.text)
            soup = BeautifulSoup(response.text, features='xml')
            print('------')
            
            weather_dict = {}
            weather_dict['areaName1'] = soup.find('areaName1').text
            
            day1 = soup.find('day')
            weather_dict['applyDate'] = day1.find('applyDate').text
            weather_dict['weather'] = day1.find('weather').text
            
            for hour1 in day1.find_all('hour'):
                print(hour1.find('time'))
                
            print(weather_dict)
            
    except Exception as e:
        print(f"Error: {e}")

In [ ]:
import httpx
import ssl
from bs4 import BeautifulSoup
from pprint import pprint

field_list = ['48160720','27230510']

for code in field_list:
    url = f"https://wwwnew.kweather.co.kr/forecast/data/lifestyle/{code}.xml"

    # SSL 설정: 보안 수준 낮춤
    context = ssl.create_default_context()
    context.set_ciphers("DEFAULT:@SECLEVEL=1")  # 보안 레벨 1로 낮춤

    try:
        with httpx.Client(verify=context, timeout=10.0) as client:
            response = client.get(url)
            response.encoding = 'utf-8'
            #print(response.text)
            soup = BeautifulSoup(response.text, features='xml')
            
        # 상단 지역 정보 추출
        area_info = {
            "areaCode": soup.find("areaCode").text,
            "areaName1": soup.find("areaName1").text,
            "areaName2": soup.find("areaName2").text,
            "areaName3": soup.find("areaName3").text,
            "announceTime": soup.find("announceTime").text,
        }

        # 날씨 데이터 리스트
        weather_data = []

        # 각 <day> 요소 순회
        for day in soup.find_all("day"):
            day_data = {
                "applyDate": day.find("applyDate").text,
                "weather": day.find("weather").text,
                "tempMin": int(day.find("tempMin").text),
                "tempMax": int(day.find("tempMax").text),
                "rainProb": int(day.find("rainProb").text),
                "am": {},
                "pm": {},
                "quarters": [],
                "hours": [],
            }

            # 오전(AM)
            am = day.find("am")
            if am:
                day_data["am"] = {
                    "weather": am.find("weather").text,
                    "tempMin": int(am.find("tempMin").text),
                    "tempMax": int(am.find("tempMax").text),
                    "rainProb": int(am.find("rainProb").text),
                }

            # 오후(PM)
            pm = day.find("pm")
            if pm:
                day_data["pm"] = {
                    "weather": pm.find("weather").text,
                    "tempMin": int(pm.find("tempMin").text),
                    "tempMax": int(pm.find("tempMax").text),
                    "rainProb": int(pm.find("rainProb").text),
                }

            # 6시간 단위 quarters
            for q in day.find_all("quarter"):
                day_data["quarters"].append({
                    "seq": int(q.find("seq").text),
                    "weather": q.find("weather").text,
                    "tempMin": int(q.find("tempMin").text),
                    "tempMax": int(q.find("tempMax").text),
                    "rainProb": int(q.find("rainProb").text),
                })

            # 시간별 hours
            for h in day.find_all("hour"):
                day_data["hours"].append({
                    "time": int(h.find("time").text),
                    "weather": h.find("weather").text,
                    "temp": int(h.find("temp").text),
                    "rainProb": int(h.find("rainProb").text),
                })

            weather_data.append(day_data)

        # 최종 결과 구조
        result = {
            "areaInfo": area_info,
            "dailyForecasts": weather_data,
        }

        # 예시 출력
        pprint(result["areaInfo"])
        pprint(result["dailyForecasts"][0])
            
    except Exception as e:
        print(f"Error: {e}")
        